In [4]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import seaborn as sns
from seaborn.distributions import distplot
import platform
from scipy import stats

In [5]:
# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

## 카이제곱 검정(chi-squared test) - 교차분석 or X^2 검정

In [6]:
# 1. 데이터 간의 연관성을 확인 할 때 사용
#    - 데이터 간 얼마나 가까운지에 대한 연관성 의미 부여
#    - 인과관계와는 무관함 (인과관계 : 특정 데이터가 다른 데이터 값의 변화에 영향을 끼치는 관계)

# 2. 카이제곱의 종류
#    - 일원카이제곱(chisquare)
#       : 1개 집단에서 기대값과 관찰값 사이의 유의미한 차이 확인
#    - 이원카이제곱(chi2_contigency)
#       : 2개 이상 집단에서 데이터 간의 연관성(차이) 확인
#       : 분석 대상의 집단 수에 의해서 "독립성 검정"과 "동질성 검정"으로 나뉜다

## 1. 일원 카이제곱
### 예시 1 - 5개의 스포츠 음료에 대한 선호도에 차이가 있는지 검정
- 귀무가설 : 스포츠 음료에 대한 선호도 차이가 없다.
- 대립가설 : 스포츠 음료에 대한 선호도 차이가 있다.

In [8]:
drink = pd.read_csv('./data/drinkdata.csv')
drink

,음료종류,관측도수
0,s1,41
1,s2,30
2,s3,51
3,s4,71
4,s5,61


In [29]:
# 검증하기
result = stats.chisquare(drink['관측도수'])
print('검정 결과 =', result)
print('검정 통계량 = %.3f, p-value = %.3f' %(result))

# 검증결과
# 1. p-value = 0.000 < 0.05 이므로 귀무가설 기각
# 2. 대립가설 채택 : 스포츠 음료에 대한 선호도 차이가 있다.

검정 결과 = Power_divergenceResult(statistic=20.488188976377952, pvalue=0.00039991784008227264)
검정 통계량 = 20.488, p-value = 0.000


## 2. 이원 카이제곱 - 독립성 검정
### 예시 2 - 교육수준과 흡연율 간의 연관성 분석
(2개집단 : 교육수준, 흡연율)
- 귀무가설 : 교육수준과 흡연율 간에 관련성이 없다.
- 대립가설 : 교육수준과 흡연율 간에 관련성이 있다.

In [21]:
smoke = pd.read_csv('./data/smoke.csv')
# 교육수준(education) : 1(대학원졸), 2(대졸), 3(고졸)
# 흡연정도(smoke) : 1.골초, 2(보통), 3(노담)
print('교육수준 :', smoke['education'].unique())
print('흡연정도 :', smoke['smoking'].unique())

교육수준 : [1 2 3]
흡연정도 : [1 2 3]


In [26]:
# 이원카이제곱 : 두 집단의 교차표 생성
# 교차표 : 빈도 집계를 의미

edu_smoke_ctab = pd.crosstab(index= smoke['education'],
                   columns= smoke['smoking'])

# 인덱스 값 한글설정
edu_smoke_ctab.index = ['대학원졸','대졸','고졸']

# 컬럼 인덱스 값 한글설정
edu_smoke_ctab.columns = ['골초','보통','노담']
edu_smoke_ctab

,골초,보통,노담
대학원졸,51,92,68
대졸,22,21,9
고졸,43,28,21


In [30]:
# 검정하기
result = stats.chi2_contingency(edu_smoke_ctab)

print('검정 결과 =', result)
print('검정 통계량 = %.3f, p-value = %.3f' % (result[0], result[1]))

# 검증결과
# 1. p-value = 0.001 < 0.05 이므로 귀무가설 기각
# 2. 대립가설 채택 : 교육수준과 흡연정도에는 관련이 있다.
#                    독립적이지 않다.

검정 결과 = (18.910915739853955, 0.0008182572832162924, 4, array([[68.94647887, 83.8056338 , 58.24788732],
       [16.9915493 , 20.65352113, 14.35492958],
       [30.06197183, 36.54084507, 25.3971831 ]]))
검정 통계량 = 18.911, p-value = 0.001


### 예시 3 - 국가전체와 지역에 대한 인종 간 인원수의 관련성 검정
(2개집단 :국가전체, 특정지역)
- 귀무가설 : 국가전체와 지역에 대한 인종 간 인원수는 관련성이 없다.
- 대립가설 : 국가전체와 지역에 대한 인종 간 인원수는 관련성이 있다.

In [37]:
# 데이터 생성

# 국가전체 데이터 생성
national = pd.DataFrame(['white'] * 100000 + ['hispanic'] * 60000 + 
                        ['black'] * 50000  + ['asian'] * 15000 +
                        ['other'] * 35000)

In [38]:
# 국가전체 데이터 생성
local = pd.DataFrame(['white'] * 600 + ['hispanic'] * 300 + 
                     ['black'] * 250  + ['asian'] * 75 +
                     ['other'] * 150)

In [51]:
# 교차표(빈도분석표) 생성
# 국가 교차표
national_ctab = pd.crosstab(index= national[0], columns='count')
national_ctab

col_0,count
0,
asian,15000
black,50000
hispanic,60000
other,35000
white,100000


In [52]:
# 지역 교차표
local_ctab = pd.crosstab(index= local[0], columns='count')
local_ctab

col_0,count
0,
asian,75
black,250
hispanic,300
other,150
white,600


In [53]:
# 데이터 합치기
national_ctab['count_loc'] = local_ctab['count']
national_ctab

col_0,count,count_loc
0,,
asian,15000,75
black,50000,250
hispanic,60000,300
other,35000,150
white,100000,600


In [81]:
# 검정하기
result = stats.chi2_contingency(national_ctab)

print('검정 결과 =', result)
print('검정 통계량 = %.3f, p-value = %.3f' % (result[0], result[1]))

# 검증결과
# 1. p-value = 0.001 < 0.05 이므로 귀무가설 기각
# 2. 대립가설 채택 : 국가전체와 지역에 대한 인종 간 인원수는 관련성이 있다.
#                    독립적이지 않다.

검정 결과 = (18.099524243141698, 0.0011800326671747886, 4, array([[1.49956958e+04, 7.93041607e+01],
       [4.99856528e+04, 2.64347202e+02],
       [5.99827834e+04, 3.17216643e+02],
       [3.49650885e+04, 1.84911526e+02],
       [1.00070780e+05, 5.29220469e+02]]))
검정 통계량 = 18.100, p-value = 0.001


## 이원카이제곱 - 동질성 검정(빈도 or 비율 데이터)
### 예시 4 - 교육방법에 따른 교육생들의 만족도 분석
(2개집단 : 교육방법, 만족도)
- 귀무가설 : 교육방법에 따른 교육생들의 만족도에 차이가 없다.(동질이다)
- 대립가설 : 교육방법에 따른 교육생들의 만족도에 차이가 있다.(동질하지 않다)

In [73]:
# 데이터 생성
survey = pd.read_csv('./data/survey_method.csv')

# survey(만족도) : 1(매우만족), 2(만족), 3(보통), 4(불만족), 5(매우불만족)
print('survey :', survey['survey'].unique())
# method(교육방법) : 1(방법1), 2(방법2), 3(방법3)
print('method :', survey['method'].unique())

survey : [1 2 3 4 5]
method : [1 2 3]


In [79]:
sur_met_ctab = pd.crosstab(index= survey['method'], columns=survey['survey'])
sur_met_ctab.index = ['방법1','방법2','방법3']
sur_met_ctab.columns = ['매우만족', '만족', '보통', '불만족', '매우불만족']
sur_met_ctab

,매우만족,만족,보통,불만족,매우불만족
방법1,5,8,15,16,6
방법2,8,14,11,11,6
방법3,8,7,11,15,9


In [97]:
# 검정하기
result = stats.chi2_contingency(sur_met_ctab)

print('검정 결과 =', result)
print('검정 통계량 = %.3f, p-value = %.3f' % (result[0], result[1]))

# 검증결과
# 1. p-value = 0.586 > 0.05 이므로 귀무가설 채택
# 2. 귀무가설 채택 : 교육방법에 따른 교육생들의 만족도에 차이가 없다.
#                    동질이다.

검정 결과 = (6.544667820529891, 0.5864574374550608, 8, array([[ 7.        ,  9.66666667, 12.33333333, 14.        ,  7.        ],
       [ 7.        ,  9.66666667, 12.33333333, 14.        ,  7.        ],
       [ 7.        ,  9.66666667, 12.33333333, 14.        ,  7.        ]]))
검정 통계량 = 6.545, p-value = 0.586


### 예시 5 - 연령대별 SNS 이용률 차이 검정
(2개집단 : 연령, SNS이용률)
- 귀무가설: 연령대별로 SNS 서비스 이용 현황에 차이가 없다.(동질하다)
- 대립가설: 연령대별로 SNS 서비스 이용 현황에 차이가 있다.(동질하지 않다)

In [91]:
# 데이터 생성
sns = pd.read_csv('./data/snsbyage.csv')

# age(연령대) : 1(청소년), 2(중년), 3(노년) 
print('연령대 :', sns['age'].unique())
# sevice(SNS서비스)
print('SNS :', sns['service'].unique())

연령대 : [1 2 3]
SNS : ['F' 'T' 'K' 'C' 'E']


In [93]:
age_sns_ctab = pd.crosstab(index=sns['age'], columns=sns['service'])
age_sns_ctab.index = ['청소년','중년','노년']
age_sns_ctab

service,C,E,F,K,T
청소년,81,16,207,111,117
중년,109,15,107,236,104
노년,32,17,78,133,76


In [96]:
# 검정하기
result = stats.chi2_contingency(age_sns_ctab)

print('검정 결과 =', result)
print('검정 통계량 = %.3f, p-value = %.3f' % (result[0], result[1]))

# 검증결과
# 1. p-value = 0.000 < 0.05 이므로 귀무가설 기각
# 2. 대립가설 채택 : 연령대별로 SNS 서비스 이용 현황에 차이가 있다.
#                    동질하지 않다.

검정 결과 = (102.75202494484225, 1.1679064204212775e-18, 8, array([[ 82.07366227,  17.74565671, 144.9228631 , 177.45656706,
        109.80125087],
       [ 88.09034051,  19.04656011, 155.54690757, 190.46560111,
        117.85059069],
       [ 51.83599722,  11.20778318,  91.53022933, 112.07783183,
         69.34815844]]))
검정 통계량 = 102.752, p-value = 0.000
